In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from fos import Supervisor, Trainer
from fos.meters import NotebookMeter

### Introduction
This notebook demonstrates how to use **FOS** to train model that has multiple inputs and target values. It uses some dummy data and a model to demonstrate the pattern to use. There are three aspects that require some extra attention:

1. The `data`, in this case a dataset. The dataset is used to return the two values for the input and the target. This is done by putting the input and target     values in their own tuple. The input and target values don't need to be of the same length, that depends on what your model expects and returns. The more generic pattern would be:

        return (x1,x2,...,xn), (y1,y2,...ym)
        
        
2. The `forward` method of the model gets the input (x) values as a tuple and can easily access them:

        x1, x2, ...., xn = x
        
        
3. Lastly the `loss` function get both the predicted values and target values as tuples and can access them 
the same way as the forward function. Here we combine two different lost function and return the sum of them. Although not 
used in this example, but the same would apply to any metrics you would like to include.

In [2]:
class Dataset(torch.utils.data.Dataset):
    '''Example dataset that returns two random values for the input 
       and two random values the target'''
    
    def __init__(self):
        super().__init__() 

    def __len__(self):
        return 1000
    
    def __getitem__(self, start):
        x1 = torch.randn(10)
        x2 = torch.randn(10)
        y1 = torch.randn(2)
        y2 = torch.randn(2)
        return (x1, x2), (y1, y2)
            

class Predictor(nn.Module):
    ''' A network with two fully connected layers and a `dummy` forward
        that just performs some arbritrary operations.
    '''
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 2)
        self.fc2 = nn.Linear(10, 2)
 
    def forward(self, x):
        x1, x2 = x
        y1 = self.fc1(x1)-self.fc2(x2)
        y2 = self.fc2(x1)+self.fc2(x2)
        return y1, y2



In [3]:
def combined_loss(pred, target):
    '''An example loss function that demonstrates how to combine two 
       different losses for the two predictions.
    ''' 
    p1, p2 = pred
    t1, t2 = target
    return F.mse_loss(p1, t1) + F.l1_loss(p2, t2)

### Setup
The rest is the same as with any other type of model. So please check the basic tutorial if this isn't clear


In [4]:
predictor = Predictor().to("cuda")
optim     = torch.optim.Adam(predictor.parameters())
data      = DataLoader(Dataset())

In [5]:
meter   = NotebookMeter()
model   = Supervisor(predictor, combined_loss)
trainer = Trainer(model, optim, meter)

### Run the training

In [6]:
trainer.run(data, epochs=10)

[  0:  1000] loss=2.32668 : 100%|██████████|00:02<00:00
[  1:  2000] loss=1.87865 : 100%|██████████|00:02<00:00
[  2:  3000] loss=1.80046 : 100%|██████████|00:02<00:00
[  3:  4000] loss=1.80939 : 100%|██████████|00:02<00:00
[  4:  5000] loss=1.83453 : 100%|██████████|00:02<00:00
[  5:  6000] loss=1.85153 : 100%|██████████|00:02<00:00
[  6:  7000] loss=1.84351 : 100%|██████████|00:02<00:00
[  7:  8000] loss=1.76520 : 100%|██████████|00:02<00:00
[  8:  9000] loss=1.83763 : 100%|██████████|00:02<00:00
[  9: 10000] loss=1.85692 : 100%|██████████|00:02<00:00
